# 📌 Dashboards Assignment: Interactive Dash Applications

## **Objective**  
Create Dash applications that demonstrate **State, multiple inputs, and multiple outputs** in interactive dashboards.

---


## **📝 Task 1: Live Text Update vs. Button-Triggered Update**
### **Requirement**
- Build a simple Dash app that includes:
  - **An input box (`dcc.Input`)** where the user types text.
  - **Two ways to update the text display:**
    1. **Live Update:** The displayed text updates dynamically as the user types.
    2. **Button-Triggered Update:** The displayed text updates **only when a button is clicked**.
### **Example Layout**
- `dcc.Input`: User enters text.
- `html.Button`: Triggers the text update (for `State` example).
- `html.H3`: Displays the entered text.

---


In [3]:
import dash
from dash import dcc, html, Input, Output, State

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H2("Live and Button-Controlled Text Display"),

    html.Label("Type something:"),
    dcc.Input(id='input-box', type='text', value='', style={'marginRight': '10px'}),

    html.Button('Update Text', id='update-button', n_clicks=0),

    html.H3("Live Update Output:"),
    html.Div(id='live-output'),

    html.H3("Button-Triggered Output:"),
    html.Div(id='button-output')
])

# Live update callback
@app.callback(
    Output('live-output', 'children'),
    Input('input-box', 'value')
)
def update_live_output(input_text):
    return input_text  # Display text live as user types

# Button-triggered update callback
@app.callback(
    Output('button-output', 'children'),
    Input('update-button', 'n_clicks'),
    State('input-box', 'value')
)
def update_on_click(n_clicks, input_text):
    return input_text if n_clicks > 0 else ''  # Display text after button click

# Run the app on port 8058
if __name__ == '__main__':
    app.run(debug=True, port=8058)

---

## **📝 Task 2: Interactive Chart with Multiple Inputs**
### **Requirement**
- Build a **data visualization dashboard** where:
  - A user selects a **country** from a dropdown menu.
  - A slider allows adjusting the **year range** for data filtering.
  - A **line chart (`dcc.Graph`)** updates dynamically to show trends based on both inputs.

### **Example Layout**
- `dcc.Dropdown`: Select a country.
- `dcc.RangeSlider`: Select a year range.
- `dcc.Graph`: Displays filtered data as a line chart.

---


In [4]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Sample data for 6 countries with trend values over the years
years = list(range(2000, 2021))
data = {
    'Year': years * 6,
    'Country': (
        ['Egypt'] * len(years) +
        ['USA'] * len(years) +
        ['Germany'] * len(years) +
        ['India'] * len(years) +
        ['China'] * len(years) +
        ['Brazil'] * len(years)
    ),
    'Value': (
        [x * 2 for x in years] +         # Egypt
        [x * 3 for x in years] +         # USA
        [x * 1.5 for x in years] +       # Germany
        [x * 2.5 for x in years] +       # India
        [x * 4 for x in years] +         # China
        [x * 1.8 for x in years]         # Brazil
    )
}

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H2("📊 Country Trends Dashboard"),

    # Dropdown for selecting a country
    html.Label("Select a country:"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': c, 'value': c} for c in df['Country'].unique()],
        value='Egypt'  # Default value
    ),

    # Range slider for selecting year range
    html.Label("Select year range:"),
    dcc.RangeSlider(
        id='year-range-slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        step=1,
        value=[2005, 2015],  # Default value
        marks={str(year): str(year) for year in range(df['Year'].min(), df['Year'].max() + 1, 5)}
    ),

    # Graph for displaying the data
    dcc.Graph(id='line-chart')
])

# Callback function to update the graph based on user input
@app.callback(
    Output('line-chart', 'figure'),
    Input('country-dropdown', 'value'),
    Input('year-range-slider', 'value')
)
def update_chart(selected_country, year_range):
    # Filter data based on selected country and year range
    filtered_df = df[
        (df['Country'] == selected_country) &
        (df['Year'] >= year_range[0]) &
        (df['Year'] <= year_range[1])
    ]
    # Create the line chart
    fig = px.line(filtered_df, x='Year', y='Value',
                  title=f"{selected_country} Data from {year_range[0]} to {year_range[1]}")
    return fig

# Run the server on port 8055
if __name__ == '__main__':
    app.run(debug=True, port=8055, use_reloader=False)


---

## **📝 Task 3: Multiple Outputs - Interactive UI Updates**
### **Requirement**
- Create an interactive dashboard where clicking a button updates **two components simultaneously**:
  - A **text component (`H3`)** displaying how many times the button has been clicked.
  - A **background color of a div (`html.Div`)**, which changes color randomly on each click.

### **Example Layout**
- `html.Button`: Click to trigger updates.
- `html.H3`: Displays the number of clicks.
- `html.Div`: Background color changes dynamically.

---


In [8]:
import dash
from dash import html, Input, Output, State
import random

# Initialize Dash app
app = dash.Dash(__name__)

# Function to generate a random hex color
def get_random_color():
    return f"#{random.randint(0, 0xFFFFFF):06x}"

# Layout
app.layout = html.Div([
    html.H2("🎨 Button Click & Color Display"),

    html.Button("Click Me!", id='click-button', n_clicks=0),

    html.H3(id='click-count', children="Button clicked 0 times"),

    html.Div(
        id='color-box',
        children='Color: #f0f0f0',  # Initial color label
        style={
            'height': '200px',
            'marginTop': '20px',
            'backgroundColor': '#f0f0f0',
            'border': '2px solid black',
            'textAlign': 'center',
            'lineHeight': '200px',
            'fontWeight': 'bold',
            'fontSize': '20px'
        }
    )
])

# Callback to update both text and box color + label
@app.callback(
    Output('click-count', 'children'),
    Output('color-box', 'style'),
    Output('color-box', 'children'),
    Input('click-button', 'n_clicks'),
    State('color-box', 'style')
)
def update_output(n_clicks, current_style):
    # Generate a random color
    new_color = get_random_color()

    # Update style
    updated_style = current_style.copy()
    updated_style['backgroundColor'] = new_color

    # Update text
    click_text = f"Button clicked {n_clicks} times"
    color_text = f"Color: {new_color}"

    return click_text, updated_style, color_text

if __name__ == '__main__':
    app.run(debug=True, port=8049, use_reloader=False)

---
# 📌 Task 4: Interactive Scatter Plot with User Controls using NumPy & Dash

## 🔹 Objective
Build an **interactive scatter plot** where users can dynamically select:
- The **X-axis feature** (e.g., numerical values from the dataset).
- The **Y-axis feature** (another numerical column).

We will generate **synthetic data** using **NumPy** instead of using built-in datasets like Iris.

## 🔹 How It Works
### **1️⃣ Generate Random Data using NumPy**
# Create a dataset with 500 random points

```python
num_samples = 500
- Feature_A: Normally distributed values around 50 with a standard deviation of 15. (X)

     np.random.normal(50, 15, num_samples)
- Feature_B: Random values uniformly distributed between 10 and 100. (Y)

     np.random.uniform(10, 100, num_samples)
- Feature_C: Random integer values between 1 and 100. (Z)                                                   
          
    np.random.randint(1, 100, num_samples)

### **2️⃣ Build an Interactive Dashboard using Dash**
- Users can choose **X and Y** or **Z and Y**  features from dropdown menus.
- A **scatter plot dynamically updates** based on the selections.

---

In [11]:
import dash
from dash import dcc, html, Input, Output
import numpy as np
import plotly.express as px
import pandas as pd

# Generate random data using NumPy
num_samples = 500
np.random.seed(42)

# Feature_A: Normally distributed around 50 with std dev of 15
feature_A = np.random.normal(50, 15, num_samples)

# Feature_B: Uniform distribution between 10 and 100
feature_B = np.random.uniform(10, 100, num_samples)

# Feature_C: Random integers between 1 and 100
feature_C = np.random.randint(1, 100, num_samples)

# Create a DataFrame with the generated features
df = pd.DataFrame({
    'Feature_A': feature_A,
    'Feature_B': feature_B,
    'Feature_C': feature_C
})

# Initialize the app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H2("📊 Interactive Scatter Plot with User Controls"),

    html.Label("Select X-axis feature:"),
    dcc.Dropdown(
        id='x-axis-dropdown',
        options=[
            {'label': 'Feature A', 'value': 'Feature_A'},
            {'label': 'Feature B', 'value': 'Feature_B'},
            {'label': 'Feature C', 'value': 'Feature_C'}
        ],
        value='Feature_A',  # Default selection
    ),

    html.Label("Select Y-axis feature:"),
    dcc.Dropdown(
        id='y-axis-dropdown',
        options=[
            {'label': 'Feature A', 'value': 'Feature_A'},
            {'label': 'Feature B', 'value': 'Feature_B'},
            {'label': 'Feature C', 'value': 'Feature_C'}
        ],
        value='Feature_B',  # Default selection
    ),

    dcc.Graph(id='scatter-plot')
])

# Callback to update the scatter plot based on the user's selections
@app.callback(
    Output('scatter-plot', 'figure'),
    Input('x-axis-dropdown', 'value'),
    Input('y-axis-dropdown', 'value')
)
def update_scatter_plot(x_axis, y_axis):
    # Create a scatter plot based on selected features
    fig = px.scatter(df, x=x_axis, y=y_axis, title=f"{x_axis} vs {y_axis}",
                     labels={x_axis: x_axis, y_axis: y_axis})
    return fig

if __name__ == '__main__':
    app.run(debug=True, port=8045, use_reloader=False)